# Demo "Relaxed Releases with DB-Workloadtests": Workload Analysis

## Disclaimer
THIS CODE AND INFORMATION ARE PROVIDED "AS IS" WITHOUT WARRANTY OF ANY KIND, EITHER EXPRESSED OR IMPLIED, INCLUDING BUT NOT LIMITED
TO THE IMPLIED WARRANTIES OF MERCHANTABILITY AND/OR FITNESS FOR
PARTICULAR PURPOSE. 

CODE IS WRITTEN FOR DEMO PURPOSES 

(C) Martin Guth and others, 2020

## Prerequisites
* SQL Server 2008 or newer (tested with SQL Server 2019 on Linux resp Docker)
* [StackOverflow-Database](https://www.brentozar.com/archive/2015/10/how-to-download-the-stack-overflow-database-via-bittorrent/) (tested with StackOverflow2010 aka small edition)
* [WorkloadTools](https://github.com/spaghettidba/WorkloadTools/releases) to analyse the workload
* Alternatively Database Clone for QueryStore and [Query Store Comparison Script](https://johnsterrett.com/2020/03/10/query-store-for-workload-replays/) by John Sterrett
* Successful completion of noteebok 3_Workload Replay respectively data in the database "WorkloadAnalysis" resulting from two replays (baseline and change)

In [1]:
-- Restore WorkloadAnalysis with Test Results (only for Live Demo :-))
RESTORE DATABASE [WorkloadAnalysis] 
FROM DISK= N'/var/opt/mssql/backup/WorkloadAnalysis_20200805.bak' 
WITH REPLACE, RECOVERY, STATS=10;

10 percent processed.

21 percent processed.

32 percent processed.

43 percent processed.

54 percent processed.

...

100 percent processed.

Processed 1184 pages for database 'WorkloadAnalysis', file 'WorkloadAnalysis' on file 1.

Processed 2 pages for database 'WorkloadAnalysis', file 'WorkloadAnalysis_log' on file 1.

RESTORE DATABASE successfully processed 1186 pages in 0.471 seconds (19.663 MB/sec).

Total execution time: 00:00:04.821

## Analysis with WorkloadViewer

Start WorkloadViewer using the following command ([Win]+[R]) to prefill the storage location for baseline and benchmark (alternatively start the program without parameters and fill them via the GUI)
```
cmd /k ""C:\Program Files\WorkloadTools\WorkloadViewer.exe"  -S . -D WorkloadAnalysis -M baseline -U sa -P P4$$w0rd! -T . -E WorkloadAnalysis -N change -V sa -Q P4$$w0rd!"

```
A description of the parameters used can be found in Gianluca's [documentation](https://github.com/spaghettidba/WorkloadTools/wiki/WorkloadViewer)

The first tab "Workload" shows a summary of the metrics cpu time, elapsed time and batches/second for the complete workloadtest.
If you click on one of the data points you can inspect the measurement. The scale is in microseconds (µs). The value 1.000.000.000 (. is used as thousands separator in Germany :-)) equals 16 minutes in total across all processor cores (queries have been running in parallel). The cyan line represents the baseline meaning the original measurement without implementing the change. The black line represents the change (called benchmark) respectively the measurements after the second replay with the change implemented.

![](./Bilder/WorkloadViewer1.png "")

On the second tab "Queries" we can see all recorded queries separately. The measurements for baseline and benchmark can be compared in a table.
The queries are normalized by WorkloadTools (meaning similar queries are summarized into one query entry...this particularly helps if you have a lot of Ad-Hoc_Statements). For further reference see Gianluca's [documentation](https://github.com/spaghettidba/WorkloadTools/wiki/SqlWorkload#analysisconsumer).

![](./Bilder/WorkloadViewer2.png "")

The last tab "QueryDetails" shows the querytext and the measurements for one query in a tabular and graphic format.

Hint: In this chart (as well as on the Tab Query Details) you can change the scale of the x-axis (Offest in minutes) by using the scroll-wheel of your mouse. Also you can zoom into a chosen timespan by pressing the right mouse button and selecting an area. The Y axis can be modified by pressing the right mouse button and moving the mouse upwards.

![](./Bilder/WorkloadViewer3.png "")

## Analysis with the Query Store Comparison Script by John Sterrett
Query Store is a great tool for more in-depth analysis of workload results. So if you are running at least SQL Server 2016 you can leverage John Sterrett's [Query Store Comparison Script](https://johnsterrett.com/2020/03/10/query-store-for-workload-replays/).

Prerequisite is that you have backed up the Query-Store-data after each execution of the workload replay (recommendation: CLONE DATABASE...see Notebook 3_WorkloadReplay for more details). Important to not is that the QueryStore of the database has just the results of one replay as we have reset the database using RESTORE before running the replay.
ook 3_Workload abspielen). 

The following script assumes that you have two databases respectively closes with the Names  "StackOverflow2010_baseline" and "StackOverflow2010_change" .

First of all we add two new schemas to our analysis database "Workload Analysis"...namely "baselineQS" and "changeQS".
In order to do that please run the following code cell twice adjusting @schemaName. The code is a bit lengthy as it ensures dropping a schema before recreating it (if it should already be present from a prior run).


In [6]:
USE [WorkloadAnalysis];
DECLARE @schemaName VARCHAR(50) = 'baselineQS'; --changeQS
DECLARE @command    NVARCHAR(200) = NULL;

-- delete tables and views
DECLARE curTruncate CURSOR LOCAL FORWARD_ONLY STATIC READ_ONLY FOR
	SELECT    
		'DROP TABLE ' + QUOTENAME(@schemaName) + '.' + QUOTENAME(t.name) + ';' AS command
	FROM   
				sys.tables t
	INNER JOIN  sys.schemas s ON t.schema_id = s.schema_id
	WHERE
		s.name = @schemaName
	UNION ALL
	SELECT    
		'DROP VIEW ' + QUOTENAME(@schemaName) + '.' + QUOTENAME(t.name) + ';' AS command
	FROM   
				sys.views t
	INNER JOIN  sys.schemas s ON t.schema_id = s.schema_id
	WHERE
		s.name = @schemaName
	;

OPEN curTruncate;
FETCH NEXT FROM curTruncate INTO @command
WHILE @@FETCH_STATUS = 0
BEGIN
    PRINT @command;
    EXECUTE(@command);
    FETCH NEXT FROM curTruncate INTO @command
END
CLOSE curTruncate;
DEALLOCATE curTruncate;

-- delete schema
SET @command = (
	SELECT	'DROP SCHEMA ' + QUOTENAME(s.name) + ';' AS command 
	FROM	sys.schemas s	
	WHERE   s.name = @schemaName
	);
PRINT @command;
EXECUTE(@command);

-- create a new schema
SET @command = 'CREATE SCHEMA ' + QUOTENAME(@schemaName);
PRINT @command;
EXECUTE(@command);



/* plausibility checks */
SELECT name FROM sys.databases WHERE name = 'WorkloadAnalysis';
SELECT name FROM WorkloadAnalysis.sys.schemas WHERE name = @schemaName;
SELECT COUNT(*) AS nrOfTables FROM WorkloadAnalysis.sys.tables t INNER JOIN  sys.schemas s ON t.schema_id = s.schema_id AND s.name = @schemaName;

CREATE SCHEMA [changeQS]

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.101

name WorkloadAnalysis

name changeQS

anzTabellen 0


Now let's import data from our snapshots to the new schemas. It would be possible to query the snapshots themselves...however by importing them we have the opportunity to create additional useful indexes. The Query-Store-Tables themselves are optimized for insert performance rather than select performance. According to John Sterrett ColumnStore indexes can help a lot with efficient data querying in this scenario.

In [7]:
/* Script originally by John Sterrett...slightly adapted for this demo
https://johnsterrett.com/2020/03/10/query-store-for-workload-replays/
*/
use [StackOverflow2010_baseline]
GO
/* Load Data */
SELECT * INTO WorkloadAnalysis.baselineQS.query_store_runtime_stats 
FROM sys.query_store_runtime_stats;

SELECT * INTO WorkloadAnalysis.baselineQS.query_store_runtime_stats_interval 
from sys.query_store_runtime_stats_interval;

select * INTO WorkloadAnalysis.baselineQS.query_store_plan 
from sys.query_store_plan;

select * INTO WorkloadAnalysis.baselineQS.query_store_query
from sys.query_store_query;

select * INTO WorkloadAnalysis.baselineQS.query_store_query_text
from sys.query_store_query_text;
/* Addition for SQL 2017 */
select * INTO WorkloadAnalysis.baselineQS.query_store_wait_stats
 from sys.query_store_wait_stats;

/* Anlage Aggregattabelle */
SELECT SUM(Count_executions) AS TotalExecutions,
SUM(Count_executions*avg_duration) AS TotalDuration,
SUM(Count_executions*avg_logical_io_reads) AS TotalReads,
SUM(Count_executions*avg_logical_io_writes) AS TotalWrites,
SUM(count_executions*avg_cpu_time) AS TotalCPU,
query_hash
INTO WorkloadAnalysis.baselineQS.QueryResults
FROM WorkloadAnalysis.baselineQS.query_store_runtime_stats rs
JOIN WorkloadAnalysis.baselineQS.query_store_plan p ON rs.plan_id = p.plan_id
JOIN WorkloadAnalysis.baselineQS.query_store_query q ON p.query_id = q.query_id
GROUP BY q.query_hash
;

use [StackOverflow2010_change]
GO
/* Load Data */
SELECT * INTO WorkloadAnalysis.changeQS.query_store_runtime_stats 
FROM sys.query_store_runtime_stats;

SELECT * INTO WorkloadAnalysis.changeQS.query_store_runtime_stats_interval 
from sys.query_store_runtime_stats_interval;

select * INTO WorkloadAnalysis.changeQS.query_store_plan 
from sys.query_store_plan;

select * INTO WorkloadAnalysis.changeQS.query_store_query
from sys.query_store_query;

select * INTO WorkloadAnalysis.changeQS.query_store_query_text
from sys.query_store_query_text;
/* Addition for SQL 2017 */
select * INTO WorkloadAnalysis.changeQS.query_store_wait_stats
 from sys.query_store_wait_stats


/* Create an aggregate table */
SELECT SUM(Count_executions) AS TotalExecutions,
SUM(Count_executions*avg_duration) AS TotalDuration,
SUM(Count_executions*avg_logical_io_reads) AS TotalReads,
SUM(Count_executions*avg_logical_io_writes) AS TotalWrites,
SUM(count_executions*avg_cpu_time) AS TotalCPU,
query_hash
INTO WorkloadAnalysis.changeQS.QueryResults
FROM WorkloadAnalysis.changeQS.query_store_runtime_stats rs
JOIN WorkloadAnalysis.changeQS.query_store_plan p ON rs.plan_id = p.plan_id
JOIN WorkloadAnalysis.changeQS.query_store_query q ON p.query_id = q.query_id
GROUP BY q.query_hash
;


Commands completed successfully.

(154 rows affected)

(9 rows affected)

(48 rows affected)

(48 rows affected)

...

(61 rows affected)

(61 rows affected)

(384 rows affected)

(59 rows affected)

Total execution time: 00:00:01.190

Add the Clustered-Column-Store-Indexes John Sterrett recommends 

Note: In my tiny demo case I don't really need it...but hey the code should help you in your real world scenarios as well 

In [13]:
DROP INDEX IF EXISTS qsp_cci ON WorkloadAnalysis.baselineQS.query_store_plan
DROP INDEX IF EXISTS qsrs_cci ON WorkloadAnalysis.baselineQS.query_store_runtime_stats;
DROP INDEX IF EXISTS qsris_cci ON WorkloadAnalysis.baselineQS.query_store_runtime_stats_interval;
DROP INDEX IF EXISTS qsq_cci ON WorkloadAnalysis.baselineQS.query_store_query;
DROP INDEX IF EXISTS qst_cci ON WorkloadAnalysis.baselineQS.query_store_query_text;
DROP INDEX IF EXISTS qsw_cci ON WorkloadAnalysis.baselineQS.query_store_wait_stats;
DROP INDEX IF EXISTS qsp_cci ON WorkloadAnalysis.changeQS.query_store_plan
DROP INDEX IF EXISTS qsrs_cci ON WorkloadAnalysis.changeQS.query_store_runtime_stats;
DROP INDEX IF EXISTS qsris_cci ON WorkloadAnalysis.changeQS.query_store_runtime_stats_interval;
DROP INDEX IF EXISTS qsq_cci ON WorkloadAnalysis.changeQS.query_store_query;
DROP INDEX IF EXISTS qst_cci ON WorkloadAnalysis.changeQS.query_store_query_text;
DROP INDEX IF EXISTS qsw_cci ON WorkloadAnalysis.changeQS.query_store_wait_stats;

CREATE CLUSTERED COLUMNSTORE INDEX qsp_cci ON WorkloadAnalysis.baselineQS.query_store_plan;
CREATE CLUSTERED COLUMNSTORE INDEX qsrs_cci ON WorkloadAnalysis.baselineQS.query_store_runtime_stats;
CREATE CLUSTERED COLUMNSTORE INDEX qsris_cci ON WorkloadAnalysis.baselineQS.query_store_runtime_stats_interval;
CREATE CLUSTERED COLUMNSTORE INDEX qsq_cci ON WorkloadAnalysis.baselineQS.query_store_query;
CREATE CLUSTERED COLUMNSTORE INDEX qst_cci ON WorkloadAnalysis.baselineQS.query_store_query_text;
CREATE CLUSTERED COLUMNSTORE INDEX qsw_cci ON WorkloadAnalysis.baselineQS.query_store_wait_stats;


CREATE CLUSTERED COLUMNSTORE INDEX qsp_cci ON WorkloadAnalysis.changeQS.query_store_plan;
CREATE CLUSTERED COLUMNSTORE INDEX qsrs_cci ON WorkloadAnalysis.changeQS.query_store_runtime_stats;
CREATE CLUSTERED COLUMNSTORE INDEX qsris_cci ON WorkloadAnalysis.changeQS.query_store_runtime_stats_interval;
CREATE CLUSTERED COLUMNSTORE INDEX qsq_cci ON WorkloadAnalysis.changeQS.query_store_query;
CREATE CLUSTERED COLUMNSTORE INDEX qst_cci ON WorkloadAnalysis.changeQS.query_store_query_text;
CREATE CLUSTERED COLUMNSTORE INDEX qsw_cci ON WorkloadAnalysis.changeQS.query_store_wait_stats;

Commands completed successfully.

Total execution time: 00:00:01.190

In [1]:
--Test Queries
SELECT TOP 10 *
FROM WorkloadAnalysis.baselineQS.QueryResults;

SELECT TOP 10 *
FROM WorkloadAnalysis.changeQS.QueryResults;

(10 rows affected)

(10 rows affected)

Total execution time: 00:00:00.047

TotalExecutions TotalDuration TotalReads TotalWrites TotalCPU query_hash 28 305444782 22755158 0 547430537 0x12844326EA63661C 9 10655985 1543673 0 28329850 0x16EBBF013E415793 28 7804312 207400 0 3811759 0x1E1E62221DC46458 9 595109642 64844930 0 855114384 0x1F08FDF7DB4F979F 1 126423 14357 0 120330 0x1F8A22B51F07A635 1 118280 3748 0 118017 0x3F6312C46409764F 2200 790605 6706 22 493860 0x439BDBF6C1B5E939 1 156063 1394 0 142981 0x5EF486BD6A1D7F0B 1 2048034 14356 0 1258252 0x6603D1199152BD76 9 78722036 14501765 0 155359749 0x6769A045EE973053

TotalExecutions TotalDuration TotalReads TotalWrites TotalCPU query_hash 1 25 0 0 25 0x0A6673EAB3C7E30F 2 560780 48189 1 1284226 0x0DC429A23CB1FC2D 28 155686105 22767852 0 379456141 0x12844326EA63661C 9 8047628 1545903 0 27083172 0x16EBBF013E415793 28 2658392 207399 0 2372233 0x1E1E62221DC46458 9 43343362 7249710 0 116280776 0x1F08FDF7DB4F979F 1 139227 14357 0 139224 0x1F8A22B51F07A635 1 196773 3748 0 96535 0x3F6312C46409764F 2200 306927 19578 54 275054 0x439BDBF6C1B5E939 2 19955 82 0 13577 0x5EA11F59E42C127F

Finally made it through the preparations....here's the final analysis script (I converted units to seconds and gigabytes for readability)
Below you can see a nice example how Azure Data Studio can help to compare numbers more easilly by going for a visualization (this is available not just for notebooks but for queries as well).


In [31]:
/* Query Store Results */
/* Skript von John Sterrett geringfügig angepasst
https://johnsterrett.com/2020/03/10/query-store-for-workload-replays/
*/
use [WorkloadAnalysis]
GO
DROP TABLE IF EXISTS #CTE;
DROP TABLE IF EXISTS #Compare;
DROP TABLE IF EXISTS #baseline;

SELECT DISTINCT  
c.TotalExecutions - b.TotalExecutions AS ExecutionDelta,
c.TotalExecutions AS CompareExecutions,
b.TotalExecutions AS BaselineExecutions,
c.TotalDuration - b.TotalDuration AS DurationDelta,
(c.TotalDuration - b.TotalDuration )/1000000.0 AS DurationDeltaSeconds,
c.TotalCPU - b.TotalCPU AS CPUDelta,
c.TotalReads - b.TotalReads AS ReadDelta,
(c.TotalReads - b.TotalReads)/128/1024 AS ReadDeltaGB, 
c.TotalWrites - b.TotalWrites AS WriteDelta,
(c.TotalWrites - b.TotalWrites)/128/1024 AS WriteDeltaGB,
c.TotalReads AS CompareReads,
b.TotalReads AS BaselineReads,
c.TotalCPU AS CompareCPU,
b.TotalCPU AS BaselineCPU,
c.TotalDuration AS CompareDuration,
b.TotalDuration AS BaselineDuration,
c.query_hash
--q.query_sql_text
INTO #CTE
FROM baselineQS.QueryResults b
JOIN changeQS.QueryResults c ON b.query_hash = c.query_hash


select COUNT(query_sql_text) AS QueryCount, 
MAX(query_sql_text) query_sql_text, 
MIN(query_id) MinQueryID, 
qsq.query_hash
INTO #Compare
from changeQS.query_store_query qsq
JOIN changeQS.query_store_query_text q 
	ON qsq.query_text_id = q.query_text_id 
where qsq.is_internal_query = 0
GROUP BY query_hash 

select COUNT(query_sql_text) AS QueryCount, 
MAX(query_sql_text) query_sql_text, 
MIN(query_id) MinQueryID, 
qsq.query_hash
INTO #Baseline
from baselineQS.query_store_query qsq
JOIN baselineQS.query_store_query_text q 
	ON qsq.query_text_id = q.query_text_id 
where qsq.is_internal_query = 0
GROUP BY query_hash 

select cte.*
, a.QueryCount AS Compare_QueryCount
, b.QueryCount AS Baseline_QueryCount
, a.MinQueryID AS Compare_MinQueryID
, b.MinQueryID AS Baseline_MinQueryID
, a.query_sql_text
FROM #CTE cte 
JOIN #Compare a on cte.query_hash = a.query_hash
JOIN #Baseline b on cte.query_hash = b.query_hash
WHERE 1=1
AND ExecutionDelta = 0
ORDER BY ReadDelta ASC


-- nochmal als Spielerei als grafik 
select 
cte.query_hash,
CAST(DurationDeltaSeconds AS INTEGER) AS DurationDeltaSeconds
FROM #CTE cte 
JOIN #Compare a on cte.query_hash = a.query_hash
JOIN #Baseline b on cte.query_hash = b.query_hash
WHERE 1=1
AND ExecutionDelta = 0
ORDER BY DurationDeltaSeconds ASC

Commands completed successfully.

(37 rows affected)

(45 rows affected)

(35 rows affected)

(16 rows affected)

(16 rows affected)

Total execution time: 00:00:00.040

ExecutionDelta CompareExecutions BaselineExecutions DurationDelta DurationDeltaSeconds CPUDelta ReadDelta ReadDeltaGB WriteDelta WriteDeltaGB CompareReads BaselineReads CompareCPU BaselineCPU CompareDuration BaselineDuration query_hash Compare_QueryCount Baseline_QueryCount Compare_MinQueryID Baseline_MinQueryID query_sql_text 0 1173 1173 -505428191 -505,428191 -2074562400 -55574768 -424,001831054688 0 0 4515 55579283 279741 2074842141 1052211 506480402 0xDAB9585A0BB813F8 1 1 5 3 (@count int)DECLARE @count INT = (SELECT COUNT(*) FROM dbo.Votes WHERE CreationDate > '20200512') + 1 0 9 9 -379034696 -379,034696 -497465975 -14500077 -110,626808166504 0 0 396 14500473 76254 497542229 120913 379155609 0x6769A045EE973053 1 1 30 28 (@UserId int)with fights as (
 select myAnswer.ParentId as Question,
 myAnswer.Score as MyScore,
 jonsAnswer.Score as JonsScore
 from Posts as myAnswer
 inner join Posts as jonsAnswer
 on jonsAnswer.OwnerUserId = 22656 and myAnswer.ParentId = jonsAnswer.ParentId
 where myAnswer.OwnerUserId = @UserId and myAnswer.PostTypeId = 2
)

select
 case
 when MyScore > JonsScore then 'You win'
 when MyScore < JonsScore then 'Jon wins'
 else 'Tie'
 end as 'Winner',
 Question as [Post Link],
 MyScore as 'My score',
 JonsScore as 'Jon''s score'
from fights 0 9 9 -178653245 -178,653245 -270131890 -7191673 -54,8681106567383 0 0 7259285 14450958 112377984 382509874 38781003 217434248 0x67701928367A1BCD 1 1 32 30 (@UserId int)select
 count(a.Id) as [Accepted Answers],
 sum(case when a.Score = 0 then 0 else 1 end) as [Scored Answers], 
 sum(case when a.Score = 0 then 1 else 0 end) as [Unscored Answers],
 sum(CASE WHEN a.Score = 0 then 1 else 0 end)*1000 / count(a.Id) / 10.0 as [Percentage Unscored]
from
 Posts q
 inner join
 Posts a
 on a.Id = q.AcceptedAnswerId
where
 a.CommunityOwnedDate is null
 and a.OwnerUserId = @UserId
 and q.OwnerUserId != @UserId
 and a.PostTypeId = 2 0 9 9 -13490828 -13,490828 -55885023 -7070 -0,0539398193359375 10 7,62939453125E-05 487018 494088 15168573 71053596 3307749 16798577 0x7401FE8FB2672F1B 1 1 19 17 select top 20 count(v.PostId) as 'Vote count', v.PostId AS [Post Link],p.Body
from Votes v 
inner join Posts p on p.Id=v.PostId
where PostTypeId = 1 and v.VoteTypeId=3
group by v.PostId,p.Body
order by 'Vote count' desc 0 9 9 -4916729 -4,916729 20036002 -3999 -0,0305099487304688 0 0 7319074 7323073 145793810 125757808 47092479 52009208 0x12844326EA63661C 1 1 26 24 SELECT 
 TOP 500
 Users.Id as [User Link],
 Count(Posts.Id) AS Answers,
 CAST(AVG(CAST(Score AS float)) as numeric(6,2)) AS [Average Answer Score]
FROM
 Posts
 INNER JOIN
 Users ON Users.Id = OwnerUserId
WHERE 
 PostTypeId = 2 and CommunityOwnedDate is null and ClosedDate is null
GROUP BY
 Users.Id, DisplayName
HAVING
 Count(Posts.Id) > 10
ORDER BY
 [Average Answer Score] DESC 0 840 840 -2329350 -2,32935 -148174 -92 -0,000701904296875 -11 -8,392333984375E-05 3393 3485 127159 275333 570369 2899719 0x8FF146B200807527 1 1 3 1 UPDATE dbo.Posts
	SET
 ViewCount = ViewCount +1
	WHERE 
 Id = 4315748 0 118 118 -51083 -0,051083 -7153 0 0 -4 -3,0517578125E-05 372 372 13455 20608 17265 68348 0xFE7A3B6DA3924B7A 1 1 9 11 (@count int)INSERT INTO	dbo.Users
	(
 Age,
 CreationDate,
 DisplayName,
 DownVotes,
 LastAccessDate,
 Reputation,
 UpVotes,
 [Views]
	)
	VALUES
	(
 37, -- Age - int
 '20200512 17:00', -- CreationDate - datetime
 'WORKLOADUSER' + CAST(@count AS VARCHAR(20)), -- DISPLAYNAME - NVARCHAR
 0, -- DownVotes - int
 '20200512 18:00', -- LastAccessDate - datetime
 @count*100, -- Reputation - int
 @count*2, -- UpVotes - int
 @count/20 -- Views - int
	) 0 9 9 -8364989 -8,364989 -2177901 0 0 0 0 133306 133306 1267278 3445179 1559942 9924931 0xA0C9265EF1C37D49 1 1 31 29 (@UserId int)WITH Rankings AS (
SELECT Id, Ranking = ROW_NUMBER() OVER(ORDER BY Reputation DESC)
FROM Users
)
,Counts AS (
SELECT Count = COUNT(*)
FROM Users
WHERE Reputation > 100
)
SELECT Id,

query_hash DurationDeltaSeconds 0xDAB9585A0BB813F8 -505 0x6769A045EE973053 -379 0x67701928367A1BCD -178 0x738C2958B0C85382 -96 0xC3F4034726CF349F -58 0x7401FE8FB2672F1B -13 0xD949A7194561CEA2 -11 0xA0C9265EF1C37D49 -8 0x12844326EA63661C -4 0xDE6AEB2BA83F2BD2 -3 0x8FF146B200807527 -2 0x1E1E62221DC46458 0 0xF49E3C4ED9C8ACE5 0 0x439BDBF6C1B5E939 0 0xFE7A3B6DA3924B7A 0 0xA1D4571BE443659F 0

WorkloadViewer suggested in my case that the number of query executions might have differed between replays:
Let's look at QueryStore if there are in fact queries which have a different number of execution

In [34]:
select cte.*
, a.QueryCount AS Compare_QueryCount
, b.QueryCount AS Baseline_QueryCount
, a.MinQueryID AS Compare_MinQueryID
, b.MinQueryID AS Baseline_MinQueryID
, a.query_sql_text
FROM #CTE cte 
JOIN #Compare a on cte.query_hash = a.query_hash
JOIN #Baseline b on cte.query_hash = b.query_hash
WHERE 1=1
AND ExecutionDelta <> 0
ORDER BY ABS(ExecutionDelta) DESC



(9 rows affected)

Total execution time: 00:00:00.021

ExecutionDelta CompareExecutions BaselineExecutions DurationDelta DurationDeltaSeconds CPUDelta ReadDelta ReadDeltaGB WriteDelta WriteDeltaGB CompareReads BaselineReads CompareCPU BaselineCPU CompareDuration BaselineDuration query_hash Compare_QueryCount Baseline_QueryCount Compare_MinQueryID Baseline_MinQueryID query_sql_text -24 1 25 -9717 -0,009717 -9703 -48 -0,0003662109375 0 0 2 50 129 9832 129 9846 0x95762C7D4908F49F 1 1 13 6 (@i int)INSERT @blockers
 (
 session_id
 )
 SELECT TOP(@i)
 blocked
 FROM @sessions
 WHERE
 NULLIF(blocked, 0) IS NOT NULL

 EXCEPT

 SELECT TOP(@i)
 session_id
 FROM @sessions -1 26 27 -946030 -0,94603 -379688 94 0,0007171630859375 -35 -0,00026702880859375 18049 17955 76199 455887 80425 1026455 0xC12A52B8ACFC2178 1 1 10 5 (@i int,@blocker bit)INSERT @sessions
 (
 session_id,
 request_id,
 login_time,
 last_request_end_time,
 status,
 statement_start_offset,
 statement_end_offset,
 sql_handle,
 host_name,
 login_name,
 program_name,
 database_id,
 memory_usage,
 open_tran_count, 
 wait_type,
 wait_resource,
 wait_time, 
 blocked,
 is_user_process,
 cmd 
 )
 SELECT TOP(@i)
 spy.session_id,
 spy.request_id,
 spy.login_time,
 spy.last_request_end_time,
 spy.status,
 spy.statement_start_offset,
 spy.statement_end_offset,
 spy.sql_handle,
 spy.host_name,
 spy.login_name,
 spy.program_name,
 spy.database_id,
 spy.memory_usage,
 spy.open_tran_count,
 spy.wait_type,
 CASE
 WHEN
 spy.wait_type LIKE N'PAGE%LATCH_%'
 OR spy.wait_type = N'CXPACKET'
 OR spy.wait_type LIKE N'LATCH[_]%'
 OR spy.wait_type = N'OLEDB' THEN
 spy.wait_resource
 ELSE
 NULL
 END AS wait_resource,
 spy.wait_time, 
 spy.blocked,
 spy.is_user_process,
 spy.cmd
 FROM
 (
 SELECT TOP(@i)
 spx.*, 
 ROW_NUMBER() OVER
 (
 PARTITION BY
 spx.session_id,
 spx.request_id
 ORDER BY
 CASE
 WHEN spx.wait_type LIKE N'LCK[_]%' THEN 
 1
 ELSE
 99
 END,
 spx.wait_time DESC,
 spx.blocked DESC
 ) AS r 
 FROM
 (
 SELECT TOP(@i)
 sp0.session_id,
 sp0.request_id,
 sp0.login_time,
 sp0.last_request_end_time,
 LOWER(sp0.status) AS status,
 CASE
 WHEN sp0.cmd = 'CREATE INDEX' THEN
 0
 ELSE
 sp0.stmt_start
 END AS statement_start_offset,
 CASE
 WHEN sp0.cmd = N'CREATE INDEX' THEN
 -1
 ELSE
 COALESCE(NULLIF(sp0.stmt_end, 0), -1)
 END AS statement_end_offset,
 sp0.sql_handle,
 sp0.host_name,
 sp0.login_name,
 sp0.program_name,
 sp0.database_id,
 sp0.memory_usage,
 sp0.open_tran_count, 
 CASE
 WHEN sp0.wait_time > 0 AND sp0.wait_type <> N'CXPACKET' THEN
 sp0.wait_type
 ELSE
 NULL
 END AS wait_type,
 CASE
 WHEN sp0.wait_time > 0 AND sp0.wait_type <> N'CXPACKET' THEN 
 sp0.wait_resource
 ELSE
 NULL
 END AS wait_resource,
 CASE
 WHEN sp0.wait_type <> N'CXPACKET' THEN
 sp0.wait_time
 ELSE
 0
 END AS wait_time, 
 sp0.blocked,
 sp0.is_user_process,
 sp0.cmd
 FROM
 (
 SELECT TOP(@i)
 sp1.session_id,
 sp1.request_id,
 sp1.login_time,
 sp1.last_request_end_time,
 sp1.status,
 sp1.cmd,
 sp1.stmt_start,
 sp1.stmt_end,
 MAX(NULLIF(sp1.sql_handle, 0x00)) OVER (PARTITION BY sp1.session_id, sp1.request_id) AS sql_handle,
 sp1.host_name,
 MAX(sp1.login_name) OVER (PARTITION BY sp1.session_id, sp1.request_id) AS login_name,
 sp1.program_name,
 sp1.database_id,
 MAX(sp1.memory_usage) OVER (PARTITION BY sp1.session_id, sp1.request_id) AS memory_usage,
 MAX(sp1.open_tran_count) OVER (PARTITION BY sp1.session_id, sp1.request_id) AS open_tran_count,
 sp1.wait_type,
 sp1.wait_resource,
 sp1.wait_time,
 sp1.blocked,
 sp1.hostprocess,
 sp1.is_user_process
 FROM
 (
 SELECT TOP(@i)
 sp2.spid AS session_id,
 CASE sp2.status
 WHEN 'sleeping' THEN
 CONVERT(INT, 0)
 ELSE
 sp2.request_id
 END AS request_id,
 MAX(sp2.login_time) AS login_time,
 MAX(sp2.last_batch) AS last_request_end_time,
 MAX(CONVERT(VARCHAR(30), RTRIM(sp2.status)) COLLATE Latin1_General_Bin2) AS status,
 MAX(CONVERT(VARCHAR(32), RTRIM(sp2.cmd)) COLLATE Latin1_Gene

Here's a short example regarding execution plans

Hint: Apparently there is currently now functionality in Azure Data Studio for rendering a query plan from a XML. Workaround: Execute the query in SSMS for a graphical view.

In [1]:
USE WorkloadAnalysis;
DECLARE
    @queryHash VARBINARY(50) = 0xDAB9585A0BB813F8;


SELECT  
    'Compare Plan' AS source,
	qsp.plan_id,
	CAST(qsp.query_plan AS XML) AS query_plan
FROM    changeQS.query_store_query qsq
INNER JOIN changeQS.query_store_plan qsp ON qsq.query_id = qsp.query_id
WHERE   query_hash = @queryHash
UNION ALL
SELECT  
    'Baseline Plan' AS source,
	qsp.plan_id,
	CAST(qsp.query_plan AS XML) AS query_plan
FROM    baselineQS.query_store_query qsq
INNER JOIN baselineQS.query_store_plan qsp ON qsq.query_id = qsp.query_id
WHERE   query_hash = @queryHash

(2 rows affected)

Total execution time: 00:00:00.192

source plan_id query_plan Compare Plan 36 <ShowPlanXML xmlns="http://schemas.microsoft.com/sqlserver/2004/07/showplan" Version="1.539" Build="15.0.4033.1"><BatchSequence><Batch><Statements><StmtSimple StatementText="DECLARE @count INT = (SELECT COUNT(*) FROM dbo.Votes WHERE CreationDate &gt; '20200512') + 1" StatementId="1" StatementCompId="4" StatementType="SELECT" StatementSqlHandle="0x09008871CA41AA809917D31327E58C9C59340000000000000000000000000000000000000000000000000000" DatabaseContextSettingsId="2" ParentObjectId="949578421" StatementParameterizationType="0" RetrievedFromCache="false" StatementSubTreeCost="0.00869687" StatementEstRows="1" SecurityPolicyApplied="false" StatementOptmLevel="FULL" QueryHash="0xDAB9585A0BB813F8" QueryPlanHash="0xDA53252E300F7BDA" StatementOptmEarlyAbortReason="GoodEnoughPlanFound" CardinalityEstimationModelVersion="150"><StatementSetOptions QUOTED_IDENTIFIER="true" ARITHABORT="false" CONCAT_NULL_YIELDS_NULL="true" ANSI_NULLS="true" ANSI_PADDING="true" ANSI_WARNINGS="true" NUMERIC_ROUNDABORT="false" /><QueryPlan CachedPlanSize="16" CompileTime="298" CompileCPU="281" CompileMemory="336"><MemoryGrantInfo SerialRequiredMemory="0" SerialDesiredMemory="0" GrantedMemory="0" MaxUsedMemory="0" /><OptimizerHardwareDependentProperties EstimatedAvailableMemoryGrant="102067" EstimatedPagesCached="51033" EstimatedAvailableDegreeOfParallelism="4" MaxCompileMemory="3958600" /><OptimizerStatsUsage><StatisticsInfo LastUpdate="2020-05-11T12:24:12.94" ModificationCount="0" SamplingPercent="3.07764" Statistics="[_WA_Sys_00000006_0BC6C43E]" Table="[Votes]" Schema="[dbo]" Database="[StackOverflow2010]" /></OptimizerStatsUsage><RelOp NodeId="0" PhysicalOp="Compute Scalar" LogicalOp="Compute Scalar" EstimateRows="1" EstimateIO="0" EstimateCPU="1e-07" AvgRowSize="11" EstimatedTotalSubtreeCost="0.00869687" Parallel="0" EstimateRebinds="0" EstimateRewinds="0" EstimatedExecutionMode="Row"><OutputList><ColumnReference Column="Expr1004" /></OutputList><ComputeScalar><DefinedValues><DefinedValue><ColumnReference Column="Expr1004" /><ScalarOperator ScalarString="[Expr1002]+(1)"><Arithmetic Operation="ADD"><ScalarOperator><Identifier><ColumnReference Column="Expr1002" /></Identifier></ScalarOperator><ScalarOperator><Const ConstValue="(1)" /></ScalarOperator></Arithmetic></ScalarOperator></DefinedValue></DefinedValues><RelOp NodeId="1" PhysicalOp="Compute Scalar" LogicalOp="Compute Scalar" EstimateRows="1" EstimateIO="0" EstimateCPU="0" AvgRowSize="11" EstimatedTotalSubtreeCost="0.00869677" Parallel="0" EstimateRebinds="0" EstimateRewinds="0" EstimatedExecutionMode="Row"><OutputList><ColumnReference Column="Expr1002" /></OutputList><ComputeScalar><DefinedValues><DefinedValue><ColumnReference Column="Expr1002" /><ScalarOperator ScalarString="CONVERT_IMPLICIT(int,[Expr1007],0)"><Convert DataType="int" Style="0" Implicit="1"><ScalarOperator><Identifier><ColumnReference Column="Expr1007" /></Identifier></ScalarOperator></Convert></ScalarOperator></DefinedValue></DefinedValues><RelOp NodeId="2" PhysicalOp="Stream Aggregate" LogicalOp="Aggregate" EstimateRows="1" EstimateIO="0" EstimateCPU="0.00191142" AvgRowSize="11" EstimatedTotalSubtreeCost="0.00869677" Parallel="0" EstimateRebinds="0" EstimateRewinds="0" EstimatedExecutionMode="Row"><OutputList><ColumnReference Column="Expr1007" /></OutputList><StreamAggregate><DefinedValues><DefinedValue><ColumnReference Column="Expr1007" /><ScalarOperator ScalarString="Count(*)"><Aggregate AggType="countstar" Distinct="0" /></ScalarOperator></DefinedValue></DefinedValues><RelOp NodeId="3" PhysicalOp="Index Scan" LogicalOp="Index Scan" EstimateRows="3184.86" EstimatedRowsRead="3184.86" EstimateIO="0.003125" EstimateCPU="0.00366035" AvgRowSize="9" EstimatedTotalSubtreeCost="0.00678535" TableCardinality="1.01434e+07" Parallel="0" EstimateRebinds="0" EstimateRewinds="0" EstimatedExecutionMode="Row"><OutputList /><IndexScan Ordered="0" ForcedIndex="0" ForceSeek="0" ForceScan="0" NoExpandHint="0" S